## Introducción a spaCy

In [44]:
from spacy.lang.es import Spanish
from spacy import displacy

En el centro de spacy se encuentra el procesador de lenguaje natural, que llamamos `nlp`. Instanciamos un procesador de la siguiente manera. Este procesador contiene reglas específicas de tokenización.

In [29]:
nlp = Spanish()

Cuando se procesa a un texto con el `nlp`, spacy crea un documento `Doc`. El documento permite acceder al texto de una manera estructurada. Este `Doc` ademàs es un iterador de Python.

In [30]:
doc = nlp('¡Hola mundo!')

In [31]:
for token in doc:
    print(token.text)

¡
Hola
mundo
!


### El objeto `Token`

Los objetos `Token` representan a los tokens en el documento, por ejemplo, una palabra o un caracter de puntuación. Para obtener un token específico, lo indexamos en el `doc`.

In [32]:
token = doc[1]

Los objetos `Tokens` también tienen varios atributos que nos permiten acceder a más información sobre los tokens.

In [33]:
print(token.text)

Hola


### El objeto `Span`

Un objeto `Span` es una sección de un documento que consiste de uno o más tokens. Es sólo una vista de un `Doc` y no contiene datos el mismo. Para crear un `Span`, se usa la notación de Python para seleccionar una sección.

In [34]:
span = doc[1:3]

In [35]:
print(span.text)

Hola mundo


### Atributos léxicos

Algunos de los atributos de los `Token`s son los siguientes. `i` es el índice del token dentro del texto. `text` devuelve el texto del token, `is_alpha`, `is_punct` y `like_num` devuelven valores booleanos indicando si el token consiste de caracteres alfanuméricos, puntuación o si _parece_ un número. Estos son atributos léxicos, porque dependen de la palabra misma y no de su contexto.

In [36]:
doc = nlp('Cuesta $10.')

In [37]:
print('Índice:     ', [token.i for token in doc])
print('Texto:     ', [token.text for token in doc])
print('is_alpha:      ', [token.is_alpha for token in doc])
print('is_punct:      ', [token.is_punct for token in doc])
print('like_num:      ', [token.like_num for token in doc])

Índice:      [0, 1, 2, 3]
Texto:      ['Cuesta', '$', '10', '.']
is_alpha:       [True, False, False, False]
is_punct:       [False, False, False, True]
like_num:       [False, False, True, False]


## Modelos estadísticos

Algunas de las cosas más interesantes que se pueden analizar son específicas a un contexto. Por ejemplo, se puede analizar si una palabra es un verbo o si un `span` de texto es el nombre de una persona.

Los modelos estadísticos le permiten a spaCy hacer predicciones en contexto. Estas predicciones son etiquetas de **part-of-speech**, **dependencias sintácticas** y **entidades nombradas**. Estos modelos están entrenados en grandes conjuntos de datos etiquetados y pueden mejorarse con más ejemplos de entrenamiento.

spacy provee de modelos pre-entrenados que se pueden descargar. Por ejemplo,

`$ python -m spacy download es_core_news_md`

para un modelo en español entrenado sobre texto de noticias.

Después se carga el modelo.

In [38]:
nlp = spacy.load('es_core_news_md')

### Prediciendo dependencias sintácticas

Usemos al modelo para hacer predicciones. En este ejemplo, vamos a usar spacy y el modelo para predecir etiquetas de **part-of-speech**, los tipos de palabras según su contexto. Una vez que cargamos el modelo, procesamos un texto.

In [39]:
doc = nlp('Ella se comió la pizza')

Para cada token en el documento, imprimimos el texto y el atributo `pos_`, el **part-of-speech** que se predice.

In [41]:
for token in doc:
    print(token.text, token.pos_)

Ella PRON
se PRON
comió VERB
la DET
pizza NOUN


En spacy, los atributos que terminan con un `_` devuelven texto, mientras que atributos sin `_` devuelven ids. Aquí, el modelo correctamente predijo que *comió* es un verbo y que *pizza* es un sustantivo.

Además de las etiquetas POS, podemos también predecir como se relacionan las palabras. Por ejemplo, si una palabra es el objeto o el objeto en una oración. El atributo `head` devuelve la equiteta del *padre* sintáctico, el token *padre* al que esta palabra está unida.

In [42]:
for token in doc:
    print(token.text, token.pos_, token.dep_, token.head.text)

Ella PRON nsubj comió
se PRON obj comió
comió VERB ROOT comió
la DET det pizza
pizza NOUN obj comió


In [45]:
displacy.render(doc, style='dep', jupyter=True, options={'distance': 90})

Algunas etiquetas

| Etiqueta | Descripción | Ejemplo |
|----------|-------------|---------|
|**nsubj** | sujeto nominal | Ella |
|**dobj**  | objeto directo | pizza|
|**det**   | determinador (artículo) | la |

La lista completa está en https://spacy.io/api/annotation

### Prediciendo entidades

Las entidades son objetos del mundo real a las que se les asigna un nombre, por ejemplo, una persona, una organización o un país. La propiedad `doc.ents` permite acceder a las entidades predichas por el modelo. Devuelve un iterador de objetos `Span`, de modo que podemos escribir el texto y la etiqueda del texto.

In [50]:
doc = nlp('Apple está buscando comprar una startup del Reino Unido por $1000 millones.')

In [51]:
for ent in doc.ents:
    print(ent.text, ent.label_)

Apple ORG
Reino Unido LOC


Para obtener deficiones de las etiquetas más comunes, puedes usar la función de ayuda `spacy.explain`. Por ejemplo, **GPE** para una entidad geopolítica no es muy intuitivo, pero `spacy.explain` puede decirte que se refiere a países, ciudades o estados.

Esta función igual incluye a etiquetas POS y etiquetas de dependencias.

In [52]:
spacy.explain('GPE')

'Countries, cities, states'

In [53]:
spacy.explain('NNP')

'noun, proper singular'

In [54]:
spacy.explain('dobj')

'direct object'

## Coincidencias basadas en reglas

Comparadas con las expresiones regulares, el `matcher` funciona con objetos `Doc` y `Token` en lugar de con `string`s.

Es más flexible: no solo puedes buscar atributos de texto sino también otros atributos de léxico. Puedes incluso definir reglas que usen las predicciones de los modelos, por ejemplo, buscar una palabra solo si es un verbo y no un adjetivo.

Los patrones de coincidencia son listas de diccionarios. Cada diccionario describe a un token. Las `keys` son los nombres de los atributos del token, asociados a sus valores esperados.

En este ejemplo, buscamos los dos tokens con texto "iPhone" y "X".

`[{'TEXT': 'iPhone'}, {'TEXT': 'X'}]`

Podemos buscar otros atributos de tokens. Buscamos dos tokens que en minúsculas sean "iphone" y "x".

`[{'LOWER': 'iphone'}, {'LOWER': 'x'}]`

Podemos incluso escribir patrones usando atributos predichos por el modelo. Aquí, buscamos tokens con el lemma "comprar", además de un sustantivo. Esto detectaría "compramos leche", por ejemplo.

`[{'LEMMA': 'comprar'}, {'POS': 'NOUN'}]`

Para usar el `matcher`, primero lo importamos desde `spacy.matcher`. El modelo ya está cargado, así como el procesador `nlp`.

In [58]:
from spacy.matcher import Matcher

Se inicia al `matcher` con el vocabulario, `nlp.vocab`.

In [59]:
matcher = Matcher(nlp.vocab)

El método `matcher.add()` permite agregar un patrón. El primer argumento es un identificador único para el patrón, el segundo argumento es una función opcional para procesar el resultado, el tercer argumento es el patrón.

In [60]:
pattern = [{'TEXT': 'iPhone'}, {'TEXT': 'X'}]
matcher.add('IPHONE_PATTERN', None, pattern)

Para buscar coincidencias en el texto con el patrón, llamamos al `matcher` en cualquier documento. Esto devuelve las coincidencias.

In [61]:
doc = nlp('Fecha de lanzamiento del nuevo iPhone X filtrada')

In [63]:
matches = matcher(doc)

Cuando usas el `Matcher` en un `Doc`, revuelve una lista de `tuple`s. Cada `tuple` consiste de tres valores: el id de la coincidencia y los índices inicial y final del `span`. Esto significa que podemos iterar sobre las coincidencias y crear un objeto `Span`.

In [64]:
for match_id, start, end in matches:
    matched_span = doc[start:end]
    print(matched_span.text)

iPhone X


El siguiente es un ejemplo de un patrón más complejo usando atributos léxicos.

Estamos buscando cinco tokens:
1. un token que consista sólo de dígitos,
2. tres tokens en mayúscula o minúscula para "fifa", "world", "cup",
3. un token que consista en puntuación.

Este token coincide con "2018 Fifa World Cup".

In [68]:
fifa_pattern = [
    {'IS_DIGIT': True},
    {'LOWER': 'fifa'},
    {'LOWER': 'world'},
    {'LOWER': 'cup'},
    {'IS_PUNCT': True}
]

In [69]:
doc = nlp('2018 FIFA World Cup: ¡Francia triunfó!')

In [71]:
matcher.add('FIFA_PATTERN', None, fifa_pattern)
fifa_matches = matcher(doc)

In [72]:
for match_id, start, end in fifa_matches:
    matched_span = doc[start:end]
    print(matched_span.text)

2018 FIFA World Cup:


En este caso, buscamos dos tokens:
1. un sustantivo
2. seguido de un verbo con el lemma "amar".

Este token hace coincidencia con "amo gatos".

In [73]:
amor_pattern = [
    {'LEMMA': 'amar', 'POS': 'VERB'},
    {'POS': 'NOUN'}
]

In [78]:
doc = nlp('Amaba perros pero ahora amo gatos más.')

In [75]:
matcher.add('amor_pattern', None, amor_pattern)
amor_matches = matcher(doc)

In [76]:
for match_id, start, end in amor_matches:
    matched_span = doc[start:end]
    print(matched_span.text)

amo gatos


Operadores y cuantificadores te permiten definir qué tan frecuente se debe buscar coincidencias con un token. Pueden agregarse con el `key` `OP`. Aquí el operador `?` hace el token `determiner` opcional, de modo que hará coincidencia con el token con el lemma "comprar", un artículo opcional y un sustantivo.

In [84]:
comprar_pattern = [
    {'LEMMA': 'comprar'},
    {'POS': 'DET', 'OP': '?'}, #opcional
    {'POS': 'NOUN'}
]

In [97]:
doc = nlp('Me compré un teléfono. Ahora estoy comprando aplicaciones.')

In [98]:
matcher.add('comprar_pattern', None, comprar_pattern)
comprar_matches = matcher(doc)

In [99]:
for match_id, start, end in comprar_matches:
    matched_span = doc[start:end]
    print(matched_span.text)

compré un teléfono
comprando aplicaciones


Operadores y cuantificadores

| Ejemplo | Descripción |
|---|---|
| `{'OP': '!'}` | Negación: coincide 0 veces |
| `{'OP': '?'}` | Opcional: coincide una o más veces |
| `{'OP': '+'}` | Coincide 1 o más veces |
| `{'OP': '*'}` | Coincide 0 o más veces |